In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
import random
import statsmodels.stats.multitest as multi
import math
from math import log

In [2]:
def rm_twoway_anova (data1, dv, within, subject):
    """
    Repeated measurement one-way ANOVA (pg.rm_anova), long data format, protein ID as index, multiple hypothesis testing corrected by Benjamini-Hochberg.
    
    data : pandas DataFrame
    DataFrame. Note that this function can also directly be used as a
    :py:class:`pandas.DataFrame` method, in which case this argument is no
    longer needed.
    
    dv : string
    Name of column containing the dependant variable (only required if
    ``data`` is in long format).
    
    within : string
    Name of column containing the within factor (only required if ``data``
    is in long format).
    If ``within`` is a single string, then compute a one-way repeated
    measures ANOVA, if ``within`` is a list with two strings,
    compute a two-way repeated measures ANOVA.
    
    subject : string
    Name of column containing the subject identifier (only required if
    ``data`` is in long format).
    
    correction : string or boolean
    If True, also return the Greenhouse-Geisser corrected p-value.
    If 'auto' (default), compute Mauchly's test of sphericity to determine
    whether the p-values needs to be corrected
    (see :py:func:`pingouin.sphericity`).
    
    detailed : boolean
    If True, return a full ANOVA table.
    """
    #df=data1.copy()
    columns = ['protein', 'Source', 'SS', 'ddof1', 'ddof2', 'MS', 'F', 'p-unc', 'p-GG-corr', 'np2', 'eps']
    scores= pd.DataFrame(columns=columns)     
    for i in list(set(data1.index)):
        df_aov = data1.loc[i]
        aov = pg.rm_anova(data = df_aov, dv = dv, within = within, subject = subject, detailed =True)
        aov['protein'] = i
        scores = scores.append(aov, sort=False)
    scores = scores.assign(new_column=lambda x:-np.log10(scores['p-unc']))
    scores = scores.rename({'new_column' : '-Log pvalue'}, axis = 1)

    #FDR correction
    reject, qvalue = multi.fdrcorrection(scores['p-unc'], alpha=0.05, method='indep')
    scores['qvalue'] = qvalue
    scores['rejected'] = reject
    
    return scores

In [3]:
def rm_oneway_anova (data1, dv, within, subject):
    """
    Repeated measurement one-way ANOVA (pg.rm_anova), long data format, protein ID as index, multiple hypothesis testing corrected by Benjamini-Hochberg.
    
    data : pandas DataFrame
    DataFrame. Note that this function can also directly be used as a
    :py:class:`pandas.DataFrame` method, in which case this argument is no
    longer needed.
    
    dv : string
    Name of column containing the dependant variable (only required if
    ``data`` is in long format).
    
    within : string
    Name of column containing the within factor (only required if ``data``
    is in long format).
    If ``within`` is a single string, then compute a one-way repeated
    measures ANOVA, if ``within`` is a list with two strings,
    compute a two-way repeated measures ANOVA.
    
    subject : string
    Name of column containing the subject identifier (only required if
    ``data`` is in long format).
    
    correction : string or boolean
    If True, also return the Greenhouse-Geisser corrected p-value.
    If 'auto' (default), compute Mauchly's test of sphericity to determine
    whether the p-values needs to be corrected
    (see :py:func:`pingouin.sphericity`).
    
    detailed : boolean
    If True, return a full ANOVA table.
    """
    #df=data1.copy()
    columns = ['protein', 'Source', 'SS', 'DF', 'MS', 'F', 'p-unc', '-Log pvalue']
    scores= []     
    for i in list(set(data1.index)):
        df_aov = data1.loc[i]
        aov = pg.rm_anova(data = df_aov, dv = dv, within = within, subject = subject, detailed =True)
        source, ss, df, ms, f, p_unc = list(aov.iloc[0, :6])
        log = -np.log10(p_unc)
        scores.append((i, source, ss, df, ms, f, p_unc, log))
    scores = pd.DataFrame(scores)
    scores.columns = columns

    #FDR correction
    reject, qvalue = multi.fdrcorrection(scores['p-unc'], alpha=0.05, method='indep')
    scores['qvalue'] = qvalue
    scores['rejected'] = reject
    return scores

### Prepare a demo dataset comprised of: 
- 3 experimental groups
- 6 participants per group 
- 4 time points
- 10 proteins

In [4]:
participants = sorted(['Participant_' + str(i) for i in range(18)] * 4)
time_points = ['tp_' + str(i) for i in range(4)] * 18
groups = sorted(['Group_' + str(i) for i in range(3)] * 24)

In [5]:
cols = ['Sample_' + str(i) for i in range(72)]
index = ['Protein_'+ str(i) for i in range(10)]
data_demo = pd.DataFrame(np.random.randint(6, 12, 720).reshape(10, 72), 
                         columns = cols, index = index)

experimental_annotation = pd.DataFrame({'Sample ID':cols, 
                                       'age':np.random.randint(40, 50, 72),
                                       'gender':np.random.randint(0, 2, 72), 
                                       })

experimental_annotation['Participant ID'] = participants
experimental_annotation['Time point'] = time_points 
experimental_annotation['Group'] = groups

In [6]:
data_demo_long = pd.melt(data_demo, value_vars = list(data_demo.columns), value_name = 'Intensity')
data_demo_long['Protein ID'] = np.tile(data_demo.index, data_demo.shape[1])
data_demo_long.set_index('Protein ID', inplace=True)
data_demo_long.rename({'variable':'Sample ID'}, axis = 1, inplace=True)

In [7]:
cols = experimental_annotation.set_index('Sample ID').columns.to_list()
for i in cols:
    df = experimental_annotation.copy()
    dict_map = dict(zip(df['Sample ID'], df[i]))
    data_demo_long[i] = data_demo_long['Sample ID'].map(dict_map)

- Your dataset has to fit the same format

In [8]:
data_demo_long.head()

,Sample ID,Intensity,age,gender,Participant ID,Time point,Group
Protein ID,,,,,,,
Protein_0,Sample_0,8,46,1,Participant_0,tp_0,Group_0
Protein_1,Sample_0,8,46,1,Participant_0,tp_0,Group_0
Protein_2,Sample_0,6,46,1,Participant_0,tp_0,Group_0
Protein_3,Sample_0,9,46,1,Participant_0,tp_0,Group_0
Protein_4,Sample_0,10,46,1,Participant_0,tp_0,Group_0


### Perform repeated measures ANOVA
- https://pingouin-stats.org/generated/pingouin.rm_anova.html

- Oneway ANOVA

In [9]:
oneway_aov = rm_oneway_anova(data1=data_demo_long, dv='Intensity', within='Time point', subject='Participant ID')
oneway_aov.head()

,protein,Source,SS,DF,MS,F,p-unc,-Log pvalue,qvalue,rejected
0,Protein_6,Time point,4.111,3,1.370,0.526,0.666458,0.176227,0.740509,False
1,Protein_4,Time point,5.153,3,1.718,0.531,0.663340,0.178264,0.740509,False
2,Protein_0,Time point,26.222,3,8.741,2.889,0.044335,1.353253,0.443350,False
3,Protein_9,Time point,9.111,3,3.037,1.245,0.303057,0.518476,0.727705,False
4,Protein_2,Time point,2.833,3,0.944,0.266,0.849677,0.070746,0.849677,False


- Twoway ANOVA

In [10]:
twoway_aov = rm_twoway_anova(data1=data_demo_long, dv='Intensity', within=['Group', 'Time point'], subject='Participant ID')
twoway_aov.head()

C:\Users\jpx667\AnacondaProjects\lib\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\jpx667\AnacondaProjects\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\jpx667\AnacondaProjects\lib\site-packages\pandas\core\frame.py:7143: RuntimeWarning: Degrees of freedom <= 0 for slice
  baseCov = np.cov(mat.T)
C:\Users\jpx667\AnacondaProjects\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\jpx667\AnacondaProjects\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\jpx667\AnacondaProjects\lib\site-packages\pingouin\distribution.py:464: UserWarning: Epsilon values might be innaccurate in two-way repeated measures design where e

,protein,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,np2,eps,-Log pvalue,qvalue,rejected
0,Protein_6,Group,40.083,2,34,20.042,-5.602,1.000000,NaN,-0.491,NaN,-0.000000,1.0,False
1,Protein_6,Time point,12.333,3,51,4.111,0.526,0.666458,0.651646,0.030,0.919,0.176227,1.0,False
2,Protein_6,Group * Time point,32.917,6,102,5.486,-1.823,1.000000,NaN,-0.120,NaN,-0.000000,1.0,False
0,Protein_4,Group,3.583,2,34,1.792,-1.193,1.000000,NaN,-0.075,NaN,-0.000000,1.0,False
1,Protein_4,Time point,15.458,3,51,5.153,0.531,0.663340,0.648917,0.030,0.921,0.178264,1.0,False
